In [ ]:
import pandas as pd
import plotly.express as px
from datetime import datetime
from supabase import create_client
import os
import dotenv

dotenv.load_dotenv()

# Initialize Supabase client
supabase_url = os.getenv("EXPO_PUBLIC_SUPABASE_URL")
supabase_key = os.getenv("EXPO_PUBLIC_SUPABASE_ANON_KEY")
supabase = create_client(supabase_url, supabase_key)

# Fetch receipts and items
response = supabase.table("receipts").select(
    "*, receipt_items(name, price, quantity)"
).execute()

# Convert to DataFrame
receipts = []
for receipt in response.data:
    receipt_items = receipt.pop('receipt_items')
    for item in receipt_items:
        row = {**receipt, **item}
        receipts.append(row)

df = pd.DataFrame(receipts)
receipts_df = df.groupby('id').first().reset_index()

# Basic statistics
print("\n=== Basic Statistics ===")
print(f"Total number of receipts: {len(df['id'].unique())}")
print(f"Total number of items: {len(df)}")
print(f"Average items per receipt: {len(df) / len(df['id'].unique()):.2f}")

print(f"Total spent: ${receipts_df['total'].sum():.2f}")
print(f"Average receipt total: ${receipts_df['total'].mean():.2f}")

# Most common items visualization
item_counts = df['name'].value_counts().head(10)
fig1 = px.bar(item_counts, 
              title='Most Common Items',
              labels={'index': 'Item Name', 'value': 'Count'})
fig1.show()

# Spending over time
daily_spending = receipts_df.groupby('timestamp')['total'].sum().reset_index()
fig2 = px.line(daily_spending, 
               x='timestamp', 
               y='total', 
               title='Daily Spending',
               labels={'timestamp': 'Date', 'total': 'Total Spent ($)'})
fig2.show()

# Store analysis
store_spending = receipts_df.groupby('store_name').agg({
    'total': ['sum', 'count'],
    'price': 'mean'
}).round(2)
store_spending.columns = ['Total Spent', 'Number of Visits', 'Average Item Price']
store_spending = store_spending.sort_values('Total Spent', ascending=False)

print("\n=== Top Stores by Spending ===")
print(store_spending.head())

In [ ]:
# Monthly spending analysis
receipts_df['month'] = pd.to_datetime(receipts_df['timestamp']).dt.strftime('%Y-%m')
monthly_spending = receipts_df.groupby('month')['total'].sum().reset_index()

fig3 = px.bar(monthly_spending,
              x='month',
              y='total',
              title='Monthly Spending',
              labels={'month': 'Month', 'total': 'Total Spent ($)'})
fig3.show()


In [ ]:
# Price trends for each item
item_prices = df.groupby(['name', 'timestamp'])['price'].mean().reset_index()

#item_prices = item_prices[item_prices['timestamp'] > '2024-01-01']

fig4 = px.line(item_prices,
               x='timestamp',
               y='price',
               color='name',
               title='Item Prices Over Time',
               labels={'timestamp': 'Date', 
                      'price': 'Price ($)',
                      'name': 'Item Name'})

# Improve readability
fig4.update_layout(
    showlegend=True,
    legend_title_text='Item Name',
    xaxis_title='Date',
    yaxis_title='Price ($)'
)

fig4.show()


In [ ]:
# Get items that appear more than 5 times
frequent_items = df['name'].value_counts()[df['name'].value_counts() > 5].index

# Filter prices for frequent items
frequent_item_prices = item_prices[item_prices['name'].isin(frequent_items)]

# Create plot
fig4b = px.line(frequent_item_prices,
                x='timestamp',
                y='price',
                color='name', 
                title='Price Trends for Frequently Purchased Items (>5 times)',
                labels={'timestamp': 'Date',
                       'price': 'Price ($)',
                       'name': 'Item Name'})

# Improve readability
fig4b.update_layout(
    showlegend=True,
    legend_title_text='Item Name',
    xaxis_title='Date',
    yaxis_title='Price ($)'
)

fig4b.show()


In [ ]:
# Filter data for eggs and mozzarella
items_to_plot = ['EIER FH RES S-L', 'BIO MOZZAR LIGHT', 'ROMARISPENTOM.', 'SKYR NATUR']
item_labels = {'EIER FH RES S-L': 'Free Range Eggs', 
              'BIO MOZZAR LIGHT': 'Organic Light Mozzarella',
              'ROMARISPENTOM.': 'Roma Tomatoes',
              'SKYR NATUR': 'Natural Skyr'}
selected_prices = item_prices[item_prices['name'].isin(items_to_plot)]
selected_prices = selected_prices[selected_prices['price'] > 0.0]

# Add readable labels for plotting
selected_prices['display_name'] = selected_prices['name'].map(item_labels)

# Create plot with larger markers and line
fig_items = px.line(selected_prices, 
                   x='timestamp', 
                   y='price',
                   color='display_name',
                   title='Price Trends of Selected Items',
                   labels={'timestamp': 'Date',
                          'price': 'Price (€)',
                          'display_name': 'Item'},
                   template='plotly_white')  # Clean white template

# Customize the layout for better readability
fig_items.update_layout(
    title_x=0.5,  # Center the title
    title_font_size=24,
    xaxis_title_font_size=16,
    yaxis_title_font_size=16,
    font_size=14,
    legend_title_text='Item',
    xaxis_title="Date",
    yaxis_title="Price (€)",
    hovermode='x unified'
)

# Add markers and customize line
fig_items.update_traces(
    mode='lines+markers',
    line=dict(width=3),
    marker=dict(size=8)
)

# Add gridlines for better readability
fig_items.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
fig_items.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')

fig_items.show()


In [ ]:
# Filter for items with price increases
item_first_last = item_prices.groupby('name').agg({
    'price': ['first', 'last']
}).reset_index()
item_first_last.columns = ['name', 'first_price', 'last_price']

# Calculate price change
item_first_last['price_change'] = item_first_last['last_price'] - item_first_last['first_price']

# Get items with price increases
increasing_items = item_first_last[item_first_last['price_change'] > 0]

# Create dataframe with just first and last points for each item
plot_data = []
for _, row in increasing_items.iterrows():
    # First point
    plot_data.append({
        'name': row['name'],
        'timestamp': item_prices[item_prices['name'] == row['name']]['timestamp'].min(),
        'price': row['first_price']
    })
    # Last point
    plot_data.append({
        'name': row['name'],
        'timestamp': item_prices[item_prices['name'] == row['name']]['timestamp'].max(),
        'price': row['last_price']
    })

plot_df = pd.DataFrame(plot_data)

# Create plot
fig5 = px.line(plot_df,
               x='timestamp',
               y='price', 
               color='name',
               title='Items with Price Increases (First and Last Points)',
               labels={'timestamp': 'Date',
                      'price': 'Price ($)',
                      'name': 'Item Name'})

# Improve readability
fig5.update_layout(
    showlegend=True,
    legend_title_text='Item Name',
    xaxis_title='Date',
    yaxis_title='Price ($)'
)

fig5.show()

# Calculate percentage increase
item_first_last['percent_increase'] = ((item_first_last['last_price'] - item_first_last['first_price']) / item_first_last['first_price'] * 100)

# Sort by percentage increase descending and get top 10
top_10_increases = item_first_last.sort_values('percent_increase', ascending=False).head(10)

# Print results
print("\nTop 10 Items by Price Increase %:")
print("----------------------------------")
for _, row in top_10_increases.iterrows():
    # Get first and last dates for this item
    first_date = item_prices[item_prices['name'] == row['name']]['timestamp'].min()
    last_date = item_prices[item_prices['name'] == row['name']]['timestamp'].max()
    # Convert dates to DD.MM.YYYY format
    first_date = pd.to_datetime(first_date).strftime('%d.%m.%Y')
    last_date = pd.to_datetime(last_date).strftime('%d.%m.%Y')
    print(f"{row['name']}: {row['percent_increase']:.1f}%")
    print(f"    First price on {first_date}")
    print(f"    Last price on {last_date}")
